In [98]:
import pandas as pd 
import numpy as np
import geopandas as gpd
from scipy.spatial import cKDTree


In [99]:
# Load spatial data
blocks = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\Administrative\block_admin.shp')  # Replace with your file
tehsils = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Maps\HP_IDS-DRR_shapefiles\hp_tehsil_final.geojson')
blocks = blocks.to_crs(epsg=7755)  # Replace with the EPSG code suitable for your region
tehsils = tehsils.to_crs(epsg=7755)

## Rural Vulnerability

In [100]:
natural_vi = pd.read_csv(r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\Rural Vulnerability\Natural Vulnerability Index Rural.csv")
human_vi = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\Rural Vulnerability\Human Vulnerability Index - AllGroups.csv')
financial_vi = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\Rural Vulnerability\Financial Vulnerability Index Rural - AllGroups.csv')
physical_vi = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\Rural Vulnerability\Physical Vulnerability Index Rural-AllGroups.csv')
social_vi = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\Rural Vulnerability\Social Vulnerability Index Rural - AllGroups.csv')
composite_vi = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\Rural Vulnerability\Composite Vulnerability Index - Rural.csv')

In [101]:
blocks = blocks.merge(natural_vi,on='FID0', how='left')
blocks = blocks.merge(human_vi,on='FID0', how='left')
blocks = blocks.merge(financial_vi,on='FID0', how='left')
blocks = blocks.merge(physical_vi,on='FID0', how='left')
blocks= blocks.merge(social_vi,on='FID0',how='left')
blocks = blocks.merge(composite_vi,on='FID0', how='left')


In [102]:
index_prefixes = ["NVIAll", "SVIAll", "CVIAll", "HVIAll","FVIAll","PVIAll"]
classes = {"02": 1, "24": 2, "46": 3, "68": 4, "810": 5}

In [103]:
for prefix in index_prefixes:
    # Filter columns for the current prefix
    prefix_columns = [col for col in blocks.columns if col.startswith(prefix)]
    
    # Calculate the composite value
    blocks[f"{prefix}_comp"] = sum(
        blocks[f"{prefix}{cls}"] * value for cls, value in classes.items()
    )

blocks

,Block,DISTRICT,FID0,geometry,NVIAll02,NVIAll24,NVIAll46,NVIAll68,NVIAll810,HVIAll02,...,CVIAll24,CVIAll46,CVIAll68,CVIAll810,NVIAll_comp,SVIAll_comp,CVIAll_comp,HVIAll_comp,FVIAll_comp,PVIAll_comp
0,Lahaul,Lahaul & Spiti,0,"POLYGON ((3662820.800 4965935.379, 3662837.962...",0.0083,0.1833,0.3917,0.2500,0.1667,0.3917,...,0.0083,0.5917,0.4000,0.0,3.3835,4.9833,3.3917,1.6752,3.9083,2.7500
1,Baijnath,Kangra,1,"POLYGON ((3679200.972 4888800.008, 3679216.368...",0.0000,0.2868,0.5000,0.1691,0.0441,0.2353,...,0.0000,0.5221,0.4779,0.0,2.9705,5.0000,3.4779,1.8162,4.1397,3.2133
2,Nurpur,Kangra,2,"POLYGON ((3605202.591 4909774.127, 3605382.552...",0.0000,0.1948,0.5195,0.2727,0.0130,0.2468,...,0.0000,0.5584,0.4416,0.0,3.1039,4.9090,3.4416,1.7532,4.0906,2.8958
3,Rait,Kangra,3,"POLYGON ((3631078.785 4896539.702, 3631096.946...",0.0000,0.2754,0.6087,0.1159,0.0000,0.1449,...,0.0000,0.4203,0.5797,0.0,2.8405,4.9855,3.5797,1.9421,4.4641,2.9276
4,Kangra,Kangra,4,"POLYGON ((3637889.955 4887730.921, 3637943.493...",0.0097,0.0583,0.6699,0.0971,0.1650,0.1650,...,0.0097,0.3398,0.6505,0.0,3.3494,5.0000,3.6408,1.9418,3.9514,3.4272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Kalpa,Kinnaur,72,"POLYGON ((3821639.598 4837526.655, 3821688.589...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Chirgaon,Shimla,73,"POLYGON ((3788358.524 4798684.801, 3788387.451...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,Nichar,Kinnaur,74,"POLYGON ((3788843.012 4842418.725, 3788872.041...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,Spiti,Lahaul & Spiti,75,"POLYGON ((3772251.425 4958372.973, 3772276.104...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
blocks.to_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\EDA\CVI_dist_test.geojson')

In [104]:
block_scores = blocks[['Block','DISTRICT','geometry','FID0','NVIAll_comp','SVIAll_comp','CVIAll_comp','HVIAll_comp','FVIAll_comp','PVIAll_comp']]
block_scores

,Block,DISTRICT,geometry,FID0,NVIAll_comp,SVIAll_comp,CVIAll_comp,HVIAll_comp,FVIAll_comp,PVIAll_comp
0,Lahaul,Lahaul & Spiti,"POLYGON ((3662820.800 4965935.379, 3662837.962...",0,3.3835,4.9833,3.3917,1.6752,3.9083,2.7500
1,Baijnath,Kangra,"POLYGON ((3679200.972 4888800.008, 3679216.368...",1,2.9705,5.0000,3.4779,1.8162,4.1397,3.2133
2,Nurpur,Kangra,"POLYGON ((3605202.591 4909774.127, 3605382.552...",2,3.1039,4.9090,3.4416,1.7532,4.0906,2.8958
3,Rait,Kangra,"POLYGON ((3631078.785 4896539.702, 3631096.946...",3,2.8405,4.9855,3.5797,1.9421,4.4641,2.9276
4,Kangra,Kangra,"POLYGON ((3637889.955 4887730.921, 3637943.493...",4,3.3494,5.0000,3.6408,1.9418,3.9514,3.4272
...,...,...,...,...,...,...,...,...,...,...
72,Kalpa,Kinnaur,"POLYGON ((3821639.598 4837526.655, 3821688.589...",72,NaN,NaN,NaN,NaN,NaN,NaN
73,Chirgaon,Shimla,"POLYGON ((3788358.524 4798684.801, 3788387.451...",73,NaN,NaN,NaN,NaN,NaN,NaN
74,Nichar,Kinnaur,"POLYGON ((3788843.012 4842418.725, 3788872.041...",74,NaN,NaN,NaN,NaN,NaN,NaN
75,Spiti,Lahaul & Spiti,"POLYGON ((3772251.425 4958372.973, 3772276.104...",75,NaN,NaN,NaN,NaN,NaN,NaN


#### Impute missing values

In [105]:
from scipy.spatial import distance
import numpy as np

# Perform IDW to impute missing values
block_scores['centroid'] = block_scores.geometry.centroid  # Calculate centroids

# Function to apply IDW for missing values
def idw_impute(block_scores, column, power=2):
    """
    Impute missing values in a column using IDW.
    Args:
        blocks: GeoDataFrame with a centroid column and the target column to impute.
        column: The name of the column to impute.
        power: The power parameter for IDW (default = 2).
    """
    for idx, row in block_scores[block_scores[column].isna()].iterrows():
        # Calculate distances to other blocks with non-missing values
        distances = block_scores.loc[~block_scores[column].isna(), 'centroid'].distance(row['centroid'])
        non_missing = block_scores.loc[~block_scores[column].isna(), column]
        
        # Compute weights
        weights = 1 / (distances ** power)
        weights[distances == 0] = 0  # Avoid division by zero

        # Impute missing value
        block_scores.at[idx, column] = np.sum(non_missing * weights) / np.sum(weights)

# Impute missing values for all composite columns
for col in [f"{prefix}_comp" for prefix in index_prefixes]:
    idw_impute(block_scores, col)



c:\Users\saura\anaconda3\envs\cdl-env\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [106]:
# Extract centroids for IDW
#block_scores = blocks[['Block', 'DISTRICT', 'geometry', 'FID0'] + [f"{prefix}_comp" for prefix in index_prefixes]]
block_scores['centroid'] = block_scores.geometry.centroid
tehsils['centroid'] = tehsils.geometry.centroid

# Prepare data for IDW
block_coords = np.array([(point.x, point.y) for point in block_scores['centroid']])
tehsil_coords = np.array([(point.x, point.y) for point in tehsils['centroid']])
kdtree = cKDTree(block_coords)

# IDW interpolation
def idw_interpolation(tehsil_points, block_values, k=5, p=2):
    interpolated_values = []
    for tehsil in tehsil_points:
        distances, indices = kdtree.query(tehsil, k=k)
        weights = 1 / distances**p
        weights[distances == 0] = 1e9  # Handle exact matches (infinite weight)
        values = block_values.iloc[indices].values
        interpolated_values.append(np.sum(weights * values) / np.sum(weights))
    return interpolated_values

# Perform IDW for each index
for col in [f"{prefix}_comp" for prefix in index_prefixes]:
    tehsils[col] = idw_interpolation(tehsil_coords, block_scores[col])

tehsils

c:\Users\saura\anaconda3\envs\cdl-env\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,District,STATE,TEHSIL,dtcode11,object_id,tehsil_area,object_id_new,timeperiod_datetime,geometry,centroid,NVIAll_comp,SVIAll_comp,CVIAll_comp,HVIAll_comp,FVIAll_comp,PVIAll_comp
0,BILASPUR,HIMACHAL PRADESH,BHARARI,030,02-030-02104,9.197340e+07,02-030,2024-08-01,"POLYGON ((3692891.894 4829904.552, 3692916.741...",POINT (3686832.169 4825446.172),3.922822,4.767183,3.156123,1.756955,3.236622,2.521387
1,BILASPUR,HIMACHAL PRADESH,BILASPUR,030,02-030-02105,1.742456e+08,02-030,2024-08-01,"POLYGON ((3706044.898 4804192.129, 3706039.688...",POINT (3698553.886 4803131.562),3.756698,4.794923,3.119642,1.748228,3.558843,2.258582
2,BILASPUR,HIMACHAL PRADESH,GHUMARWIN,030,02-030-02106,1.949023e+08,02-030,2024-08-01,"POLYGON ((3697948.587 4820771.479, 3697962.908...",POINT (3691424.153 4816473.496),4.068299,4.785038,3.181901,1.799425,3.316268,2.329822
3,BILASPUR,HIMACHAL PRADESH,JHANDUTA,030,02-030-02107,3.278236e+08,02-030,2024-08-01,"POLYGON ((3673945.104 4817319.287, 3673967.357...",POINT (3681800.868 4806366.255),4.111813,4.889313,3.150822,1.838621,3.489056,2.189798
4,BILASPUR,HIMACHAL PRADESH,NAMHOL,030,02-030-02108,1.058904e+08,02-030,2024-08-01,"POLYGON ((3705961.853 4802017.472, 3705965.105...",POINT (3703469.539 4796880.461),3.383070,4.880515,3.210889,1.762393,3.856250,2.431787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,KINNAUR,HIMACHAL PRADESH,HANGRANG,034,02-034-01703,5.100797e+08,02-034,2024-08-01,"POLYGON ((3882916.210 4861947.043, 3882319.551...",POINT (3869021.422 4870230.141),3.339733,4.846386,3.088180,1.658286,3.353939,2.391966
108,KINNAUR,HIMACHAL PRADESH,MORANG,034,02-034-01704,1.789266e+09,02-034,2024-08-01,"POLYGON ((3824686.593 4858716.451, 3824706.555...",POINT (3858844.257 4826178.740),3.330868,4.853209,3.080351,1.653799,3.342266,2.377180
109,KINNAUR,HIMACHAL PRADESH,POOH,034,02-034-01705,1.242480e+09,02-034,2024-08-01,"POLYGON ((3855584.006 4866594.406, 3855659.661...",POINT (3859192.664 4851530.949),3.219263,4.919581,3.071895,1.622983,3.320808,2.347113
110,KINNAUR,HIMACHAL PRADESH,SANGLA,034,02-034-01706,1.457672e+09,02-034,2024-08-01,"POLYGON ((3825550.531 4820600.893, 3825938.864...",POINT (3864128.389 4796898.540),3.526562,4.741143,3.112859,1.712724,3.407312,2.452991


In [94]:
tehsil_data = tehsils[['object_id', 'geometry'] + [f"{prefix}_comp" for prefix in index_prefixes]]

# Save to file
tehsil_data.to_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\EDA\tehsil_data_with_IDW.geojson', driver='GeoJSON')


In [107]:
tehsil_variables = tehsils[['object_id','TEHSIL','District'] + [f"{prefix}_comp" for prefix in index_prefixes]]
tehsil_variables.to_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\variables\rural_vul.csv')

In [65]:
# Perform spatial overlay to get intersection areas
intersection = gpd.overlay(block_scores, tehsils, how='intersection')

# Calculate area of each intersection
intersection['area'] = intersection.geometry.area
intersection

,Block,DISTRICT,FID0,NVIAll_comp,SVIAll_comp,CVIAll_comp,HVIAll_comp,FVIAll_comp,PVIAll_comp,centroid,District,STATE,TEHSIL,dtcode11,object_id,tehsil_area,object_id_new,timeperiod_datetime,geometry,area
0,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),CHAMBA,HIMACHAL PRADESH,BRAHMAUR,023,02-023-00002,1.655486e+09,02-023,2024-08-01,"MULTIPOLYGON (((3707378.311 4930098.401, 37073...",6.955448e+05
1,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),CHAMBA,HIMACHAL PRADESH,CHAURAH,023,02-023-00004,1.163670e+09,02-023,2024-08-01,"MULTIPOLYGON (((3668814.692 4948643.327, 36687...",1.527682e+06
2,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),CHAMBA,HIMACHAL PRADESH,PANGI,023,02-023-00006,1.511085e+09,02-023,2024-08-01,"MULTIPOLYGON (((3698692.817 4977351.789, 36987...",5.163088e+06
3,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),KANGRA,HIMACHAL PRADESH,MULTHAN,024,02-024-02125,9.850664e+08,02-024,2024-08-01,"MULTIPOLYGON (((3720123.506 4920366.678, 37200...",8.059547e+03
4,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),KULLU,HIMACHAL PRADESH,BHUNTAR,026,02-026-00013,2.113375e+09,02-026,2024-08-01,"MULTIPOLYGON (((3783405.246 4885530.485, 37833...",1.847964e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,Spiti,Lahaul & Spiti,75,3.536928,4.706239,3.142344,1.728967,3.466268,2.533611,POINT (3818461.105 4909352.026),KINNAUR,HIMACHAL PRADESH,POOH,034,02-034-01705,1.242480e+09,02-034,2024-08-01,"MULTIPOLYGON (((3854393.937 4865888.601, 38543...",3.287927e+04
547,Spiti,Lahaul & Spiti,75,3.536928,4.706239,3.142344,1.728967,3.466268,2.533611,POINT (3818461.105 4909352.026),LAHAUL & SPITI,HIMACHAL PRADESH,SPITI,025,02-025-02401,7.243042e+09,02-025,2024-08-01,"POLYGON ((3774162.910 4957772.314, 3774188.109...",7.040638e+09
548,Indora,Kangra,76,3.201857,4.623953,3.169905,1.768642,3.584224,2.654598,POINT (3602014.989 4896668.765),KANGRA,HIMACHAL PRADESH,FATEHPUR,024,02-024-02116,2.778485e+08,02-024,2024-08-01,"MULTIPOLYGON (((3614859.602 4892492.273, 36149...",5.736690e+06
549,Indora,Kangra,76,3.201857,4.623953,3.169905,1.768642,3.584224,2.654598,POINT (3602014.989 4896668.765),KANGRA,HIMACHAL PRADESH,INDORA,024,02-024-02118,3.274405e+08,02-024,2024-08-01,"POLYGON ((3588372.946 4887788.748, 3588251.030...",2.642957e+08


In [66]:
# Calculate weights based on block areas
block_areas = blocks[['FID0', 'geometry']]
block_areas['block_area'] = block_areas.geometry.area
intersection = intersection.merge(block_areas[['FID0', 'block_area']], on='FID0')
intersection['weight'] = intersection['area'] / intersection['block_area']
intersection

c:\Users\saura\anaconda3\envs\cdl-env\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Block,DISTRICT,FID0,NVIAll_comp,SVIAll_comp,CVIAll_comp,HVIAll_comp,FVIAll_comp,PVIAll_comp,centroid,...,TEHSIL,dtcode11,object_id,tehsil_area,object_id_new,timeperiod_datetime,geometry,area,block_area,weight
0,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),...,BRAHMAUR,023,02-023-00002,1.655486e+09,02-023,2024-08-01,"MULTIPOLYGON (((3707378.311 4930098.401, 37073...",6.955448e+05,6.615486e+09,1.051389e-04
1,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),...,CHAURAH,023,02-023-00004,1.163670e+09,02-023,2024-08-01,"MULTIPOLYGON (((3668814.692 4948643.327, 36687...",1.527682e+06,6.615486e+09,2.309251e-04
2,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),...,PANGI,023,02-023-00006,1.511085e+09,02-023,2024-08-01,"MULTIPOLYGON (((3698692.817 4977351.789, 36987...",5.163088e+06,6.615486e+09,7.804549e-04
3,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),...,MULTHAN,024,02-024-02125,9.850664e+08,02-024,2024-08-01,"MULTIPOLYGON (((3720123.506 4920366.678, 37200...",8.059547e+03,6.615486e+09,1.218285e-06
4,Lahaul,Lahaul & Spiti,0,3.383500,4.983300,3.391700,1.675200,3.908300,2.750000,POINT (3734465.683 4945839.062),...,BHUNTAR,026,02-026-00013,2.113375e+09,02-026,2024-08-01,"MULTIPOLYGON (((3783405.246 4885530.485, 37833...",1.847964e+03,6.615486e+09,2.793391e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,Spiti,Lahaul & Spiti,75,3.536928,4.706239,3.142344,1.728967,3.466268,2.533611,POINT (3818461.105 4909352.026),...,POOH,034,02-034-01705,1.242480e+09,02-034,2024-08-01,"MULTIPOLYGON (((3854393.937 4865888.601, 38543...",3.287927e+04,7.082175e+09,4.642539e-06
547,Spiti,Lahaul & Spiti,75,3.536928,4.706239,3.142344,1.728967,3.466268,2.533611,POINT (3818461.105 4909352.026),...,SPITI,025,02-025-02401,7.243042e+09,02-025,2024-08-01,"POLYGON ((3774162.910 4957772.314, 3774188.109...",7.040638e+09,7.082175e+09,9.941350e-01
548,Indora,Kangra,76,3.201857,4.623953,3.169905,1.768642,3.584224,2.654598,POINT (3602014.989 4896668.765),...,FATEHPUR,024,02-024-02116,2.778485e+08,02-024,2024-08-01,"MULTIPOLYGON (((3614859.602 4892492.273, 36149...",5.736690e+06,4.193314e+08,1.368056e-02
549,Indora,Kangra,76,3.201857,4.623953,3.169905,1.768642,3.584224,2.654598,POINT (3602014.989 4896668.765),...,INDORA,024,02-024-02118,3.274405e+08,02-024,2024-08-01,"POLYGON ((3588372.946 4887788.748, 3588251.030...",2.642957e+08,4.193314e+08,6.302788e-01


In [67]:
# Apply weights to block-level data
#block_data = pd.read_csv('block_data.csv')  # Your block-level dataset
#intersection = intersection.merge(block_data, on='block_id')
indexes = ['NVIAll_comp', 'SVIAll_comp','CVIAll_comp', 'HVIAll_comp', 'FVIAll_comp', 'PVIAll_comp']

for col in indexes:  # Columns to aggregate
    intersection[col] = intersection[col] * intersection['weight']

# Aggregate to tehsil level
tehsil_data = intersection.groupby('object_id')[indexes].sum()
tehsil_data = tehsil_data.reset_index()
tehsil_data


,object_id,NVIAll_comp,SVIAll_comp,CVIAll_comp,HVIAll_comp,FVIAll_comp,PVIAll_comp
0,02-023-00001,1.262346,1.959686,1.209478,0.668264,1.560788,0.659706
1,02-023-00002,3.636703,5.020202,3.704563,2.048632,4.380706,2.790890
2,02-023-00003,6.181473,9.863810,6.215569,3.359532,7.610617,3.768747
3,02-023-00004,2.598382,4.966883,3.096594,1.762834,4.086206,2.407667
4,02-023-00005,0.789919,1.228995,0.759342,0.420242,0.977688,0.416246
...,...,...,...,...,...,...,...
107,02-034-00010,3.542152,4.572230,3.032887,1.691070,3.299232,2.409558
108,02-034-01703,0.451423,0.701783,0.435499,0.228833,0.470193,0.331450
109,02-034-01704,1.637841,2.537880,1.576734,0.829364,1.702794,1.200836
110,02-034-01705,1.121332,1.744863,1.082335,0.568553,1.168407,0.823451


## full dataset

In [3]:
full = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\To_sift.geojson')
full

,ID,BLOCK_ID,Block,DISTRICT,PGAMax100,PGAMax200,PGAMax2500,PGAMax500,PGAMax50,PGAMax25,...,Ru_Litrcy,Ma_Litrcy,Fe_Litrcy,FeLitrcyto,Fe_wo_Fmto,CL_to_AL,In_Wor_MW,Ot_Wor_TMW,Po_Density,geometry
0,1,1,Lahaul,Lahaul & Spiti,0.07,0.10,0.32,0.16,0.05,0.03,...,74.97,84.59,64.50,41.19,45.51,98.05,1.35,26.08,3,"MULTIPOLYGON (((76.37192 32.81328, 76.37209 32..."
1,2,3,Baijnath,Kangra,0.08,0.11,0.35,0.18,0.05,0.03,...,81.40,89.12,73.68,73.68,29.30,88.75,2.18,62.85,72,"MULTIPOLYGON (((76.56831 32.11573, 76.56846 32..."
2,3,10,Nurpur,Kangra,0.07,0.11,0.35,0.17,0.05,0.03,...,85.67,90.90,80.39,80.39,18.13,88.93,2.54,61.32,312,"MULTIPOLYGON (((75.77042 32.28824, 75.77231 32..."
3,4,13,Rait,Kangra,0.07,0.10,0.33,0.16,0.05,0.03,...,83.24,90.38,76.34,76.34,20.85,87.55,2.66,75.76,230,"MULTIPOLYGON (((76.05146 32.17469, 76.05165 32..."
4,5,6,Kangra,Kangra,0.07,0.10,0.31,0.15,0.04,0.03,...,87.44,93.99,81.00,81.00,19.63,85.35,4.24,75.25,376,"MULTIPOLYGON (((76.12691 32.09626, 76.12747 32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,1,Kalpa,Kinnaur,0.08,0.12,0.37,0.18,0.05,0.04,...,81.30,88.51,72.26,72.26,34.09,92.98,1.53,44.34,18,"MULTIPOLYGON (((78.09916 31.67333, 78.09967 31..."
73,74,9,Chirgaon,Shimla,0.08,0.12,0.39,0.19,0.06,0.04,...,69.73,79.48,59.33,59.33,34.87,87.75,2.15,18.30,53,"MULTIPOLYGON (((77.75114 31.31474, 77.75142 31..."
74,75,3,Nichar,Kinnaur,0.08,0.11,0.36,0.18,0.05,0.03,...,79.59,86.93,70.09,70.09,34.89,96.10,1.03,40.31,24,"MULTIPOLYGON (((77.74886 31.71351, 77.74916 31..."
75,76,0,Spiti,Lahaul & Spiti,0.07,0.10,0.29,0.15,0.05,0.03,...,79.76,87.37,70.74,70.74,35.45,91.98,0.24,68.86,2,"MULTIPOLYGON (((77.55052 32.76603, 77.55078 32..."


## statistical tests

In [95]:
#tehsil_output = pd.read_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\variables\rural_vul.csv')
tehsil_gdf = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\EDA\tehsil_data_with_IDW.geojson')
block = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\HP\flood-data-ecosystem-Himachal-Pradesh\Sources\HPSDMA\data\Taru HRVA datasets\EDA\CVI_dist_test.geojson')

In [78]:
tehsil_moran = pd.merge(tehsil_output,tehsil_gdf, on='object_id', how='left')
tehsil_moran

,object_id,NVIAll_comp,SVIAll_comp,CVIAll_comp,HVIAll_comp,FVIAll_comp,PVIAll_comp,District,STATE,TEHSIL,dtcode11,tehsil_area,object_id_new,timeperiod_datetime,geometry
0,02-023-00001,1.262346,1.959686,1.209478,0.668264,1.560788,0.659706,CHAMBA,HIMACHAL PRADESH,BHATTIYAT,023,2.624478e+08,02-023,2024-08-01,"POLYGON ((76.14616 32.42028, 76.14575 32.41980..."
1,02-023-00002,3.636703,5.020202,3.704563,2.048632,4.380706,2.790890,CHAMBA,HIMACHAL PRADESH,BRAHMAUR,023,1.655486e+09,02-023,2024-08-01,"POLYGON ((76.86595 32.46337, 76.86620 32.46288..."
2,02-023-00003,6.181473,9.863810,6.215569,3.359532,7.610617,3.768747,CHAMBA,HIMACHAL PRADESH,CHAMBA,023,1.008207e+09,02-023,2024-08-01,"POLYGON ((76.42627 32.59980, 76.42604 32.59934..."
3,02-023-00004,2.598382,4.966883,3.096594,1.762834,4.086206,2.407667,CHAMBA,HIMACHAL PRADESH,CHAURAH,023,1.163670e+09,02-023,2024-08-01,"POLYGON ((76.23815 33.02083, 76.23806 33.02047..."
4,02-023-00005,0.789919,1.228995,0.759342,0.420242,0.977688,0.416246,CHAMBA,HIMACHAL PRADESH,DALHOUSIE,023,1.620989e+08,02-023,2024-08-01,"POLYGON ((75.99648 32.60022, 75.99804 32.59904..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,02-034-00010,3.542152,4.572230,3.032887,1.691070,3.299232,2.409558,KINNAUR,HIMACHAL PRADESH,NICHAR,034,1.128771e+09,02-034,2024-08-01,"POLYGON ((78.07635 31.78311, 78.07678 31.78210..."
108,02-034-01703,0.451423,0.701783,0.435499,0.228833,0.470193,0.331450,KINNAUR,HIMACHAL PRADESH,HANGRANG,034,5.100797e+08,02-034,2024-08-01,"POLYGON ((78.74985 31.90206, 78.74348 31.90203..."
109,02-034-01704,1.637841,2.537880,1.576734,0.829364,1.702794,1.200836,KINNAUR,HIMACHAL PRADESH,MORANG,034,1.789266e+09,02-034,2024-08-01,"POLYGON ((78.12863 31.86676, 78.12885 31.86658..."
110,02-034-01705,1.121332,1.744863,1.082335,0.568553,1.168407,0.823451,KINNAUR,HIMACHAL PRADESH,POOH,034,1.242480e+09,02-034,2024-08-01,"POLYGON ((78.45749 31.94192, 78.45830 31.94181..."


In [96]:
from esda.moran import Moran
from libpysal.weights import Queen

# Block-level Moran's I
block_weights = Queen.from_dataframe(block)
for col in indexes:
    moran_block = Moran(block_scores[col], block_weights)
    print(f"Block-level Moran's I for {col}: {moran_block.I}, p-value: {moran_block.p_sim}")

# Tehsil-level Moran's I
tehsil_weights = Queen.from_dataframe(tehsil_moran)
for col in indexes:
    moran_tehsil = Moran(tehsil_data[col], tehsil_weights)
    print(f"Tehsil-level Moran's I for {col}: {moran_tehsil.I}, p-value: {moran_tehsil.p_sim}")

C:\Users\saura\AppData\Local\Temp\ipykernel_33468\2416944738.py:5: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  block_weights = Queen.from_dataframe(block)


Block-level Moran's I for NVIAll_comp: 0.19844216339388243, p-value: 0.002
Block-level Moran's I for SVIAll_comp: 0.08775975698989502, p-value: 0.075
Block-level Moran's I for CVIAll_comp: 0.1565641517341926, p-value: 0.013
Block-level Moran's I for HVIAll_comp: 0.009192614682233799, p-value: 0.386
Block-level Moran's I for FVIAll_comp: 0.291562390057624, p-value: 0.001
Block-level Moran's I for PVIAll_comp: 0.11375500924163326, p-value: 0.047


C:\Users\saura\AppData\Local\Temp\ipykernel_33468\2416944738.py:11: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  tehsil_weights = Queen.from_dataframe(tehsil_moran)


Tehsil-level Moran's I for NVIAll_comp: 0.135698641693524, p-value: 0.014
Tehsil-level Moran's I for SVIAll_comp: 0.014065684014784512, p-value: 0.309
Tehsil-level Moran's I for CVIAll_comp: 0.019426094456188203, p-value: 0.317
Tehsil-level Moran's I for HVIAll_comp: -0.06719754272612838, p-value: 0.154
Tehsil-level Moran's I for FVIAll_comp: 0.04805779580706997, p-value: 0.176
Tehsil-level Moran's I for PVIAll_comp: 0.017268682911498034, p-value: 0.308
